In [ ]:
import re 
import pandas as pd

In [ ]:
f=open('file name')

In [ ]:
data=f.read()

In [ ]:
print(data)

In [ ]:
pattern='\d{2}/\d{2}/\d{2},\s\d{1,2}:\d{2}\s[ap]m\s-\s'

In [ ]:
messages=re.split(pattern,data)[1:]
messages

In [ ]:
dates=re.findall(pattern,data)
dates

In [ ]:
df=pd.DataFrame({'user_message':messages,'message_date':dates})
#convert message_date type
df['message_date']=pd.to_datetime(df['message_date'], format='%d/%m/%y, %I:%M\u202f%p - ')
df.rename(columns={'message_date': 'date'},inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
# separate users and messages
users=[]
messages=[]
for message in df['user_message']:
    entry=re.split('([\w\W]+?):\s',message)
    if entry[1:]: #user name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
df['user']=users
df['message']=messages
df.drop(columns=['user_message'],inplace=True)
df.head()

In [ ]:
df['year']=df['date'].dt.year

In [ ]:
df.head()

In [ ]:
df['month']=df['date'].dt.month_name()

In [ ]:
df.head()

In [ ]:
df['day']=df['date'].dt.day

In [ ]:
df['hour']=df['date'].dt.hour

In [ ]:
df['minute']=df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df[df['user']=='Arsh Gupta'].shape

In [ ]:
words=[]
for message in df['message']:
    words.extend(message.split())
    print(message)

In [ ]:
!pip install urlextract
from urlextract import URLExtract
extractor =URLExtract()
urls=extractor.find_urls("Let's www.gmail.com have URL stackoverflow.com as an example google.com,http://facebook.com")
links=[]

for message in df['message']:
    links.extend(extractor.find_urls(message))

In [ ]:
links=[]

for message in df['message']:
    links.extend(extractor.find_urls(message))

In [ ]:
len(links)

In [ ]:
x=df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name=x.index
count=x.values
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index':'name','user':'percent'})

In [ ]:
# top 25 words used in a group


In [ ]:
words=[]
for message in df['message']:
    words.extend(message.split())

In [ ]:
temp=df[df['user']!='group_notification']
temp=temp[temp['message'] != '<Media omitted>\n']

In [ ]:
f=open('stop_hinglish.txt','r')
stop_words=f.read()
print(stop_words)

In [ ]:
words=[]
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# remove media omitted message
#remove group message
#remove stop words

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
emojis=[]
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.UNICODE_EMOJI['en']])

In [ ]:
import emoji

emojis = []
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.UNICODE_EMOJI])
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [ ]:
import emoji

emojis = []
for message in df['message']:
    for c in message:
        if any(char in emoji.UNICODE_EMOJI_ENGLISH for char in emoji.demojize(c)):
            emojis.append(c)

In [ ]:
import re 
import pandas as pd

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
emojis=[]
for message in df['message']:
    emojis.extend([c for c in message if c in emoji.UNICODE_EMOJI['en']])

In [ ]:
df['month_num']=df['date'].dt.month

In [ ]:
df

In [ ]:
timeline=df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [ ]:
timeline

In [ ]:
for i in range(timeline.shape[0]):
    print(i)

In [ ]:
for i in range(timeline.shape[0]):
    print(timeline['month'][i])

In [ ]:
time=[]
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i]+"-"+str(timeline['year'][i]))

In [ ]:
time

In [ ]:
timeline['time']=time

In [ ]:
timeline

In [ ]:
plt.plot(timeline['time'],timeline['message'])

In [ ]:
df['date'].dt.date

In [ ]:
df['only_date']=df['date'].dt.date

In [ ]:
daily_timeline=df.groupby('only_date').count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'],daily_timeline['message'])

In [ ]:
df['day_name']=df['date'].dt.day_name()

In [ ]:
day_timeline=df.groupby('day_name').count()['message'].reset_index()

In [ ]:
plt.figure(figsize=(18,10))
plt.plot(day_timeline['day_name'],day_timeline['message'])

In [ ]:
df['day_name'].value_counts()

In [ ]:
df.head()

In [ ]:
period=[]
for hour in df[['day_name','hour']]['hour']:
    if hour==23:
        period.append(str(hour)+"-"+str('00'))
    elif hour==0:
        period.append(str('00')+"-"+str(hour+1))
    else:
        period.append(str(hour)+"-"+str(hour+1))

In [ ]:
df['period']=period

In [ ]:
df.sample(5)

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0))
plt.yticks(rotation='horizontal')
plt.show()

In [ ]:
df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0)